In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [2]:
# ---------------------------
# Dataset Paths
# ---------------------------
train_dir = "E:/VS/Project/DERMNET/train"
test_dir = "E:/VS/Project/DERMNET/test"

In [3]:
# Dynamically get class labels from folders
label_names = sorted(os.listdir(train_dir))
label_map = {label: index for index, label in enumerate(label_names)}
print("Classes found:", label_names)

Classes found: ['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


In [4]:
# ---------------------------
# Load and preprocess images
# ---------------------------
def load_data(data_dir, img_size=(128, 128)):
    images = []
    labels = []

    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                img = cv2.imread(img_path)  # Read in color (BGR)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    if label in label_map:
                        labels.append(label_map[label])
                else:
                    print(f"Warning: Could not load {img_path}")

    images = np.array(images, dtype="float32") / 255.0
    labels = to_categorical(np.array(labels), num_classes=len(label_names))
    return images, labels

In [5]:
print("Train dir exists:", os.path.exists(train_dir))
print("Subfolders in train_dir:", os.listdir(train_dir))

Train dir exists: True
Subfolders in train_dir: ['Acne and Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Atopic Dermatitis Photos', 'Bullous Disease Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos', 'Exanthems and Drug Eruptions', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos', 'Light Diseases and Disorders of Pigmentation', 'Lupus and other Connective Tissue diseases', 'Melanoma Skin Cancer Nevi and Moles', 'Nail Fungus and other Nail Disease', 'Poison Ivy Photos and other Contact Dermatitis', 'Psoriasis pictures Lichen Planus and related diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Seborrheic Keratoses and other Benign Tumors', 'Systemic Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Urticaria Hives', 'Vascular Tumors', 'Vasculitis Photos', 'Warts Molluscum and other Viral Infections']


In [6]:
# Load datasets
train_images, train_labels = load_data(train_dir)
test_images, test_labels = load_data(test_dir)

In [ ]:
# Train/Validation split
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

In [ ]:
# ---------------------------
# Data Augmentation
# ---------------------------
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
# ---------------------------
# CNN Model Architecture
# ---------------------------
model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),

    layers.Conv2D(32, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(256, (3, 3), activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(len(label_names), activation="softmax")
])

# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# ---------------------------
# Callbacks
# ---------------------------
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.5, min_lr=1e-7, verbose=1)
model_checkpoint = ModelCheckpoint("best_dermnet_model.keras", monitor="val_loss", save_best_only=True, verbose=1)

In [ ]:
# ---------------------------
# Training
# ---------------------------
history = model.fit(
    datagen.flow(train_images, train_labels, batch_size=32),
    validation_data=(val_images, val_labels),
    epochs=50,
    callbacks=[early_stopping, lr_reduction, model_checkpoint],
    verbose=1
)

C:\Users\Naveen Singh Rawat\AppData\Roaming\Python\Python313\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
389/389 ━━━━━━━━━━━━━━━━━━━━ 0s 971ms/step - accuracy: 0.1030 - loss: 4.8876
Epoch 1: val_loss improved from None to 4.42262, saving model to best_dermnet_model.keras
389/389 ━━━━━━━━━━━━━━━━━━━━ 406s 1s/step - accuracy: 0.1194 - loss: 4.4652 - val_accuracy: 0.1292 - val_loss: 4.4226 - learning_rate: 1.0000e-04
Epoch 2/50
389/389 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1467 - loss: 4.1833
Epoch 2: val_loss improved from 4.42262 to 4.00513, saving model to best_dermnet_model.keras
389/389 ━━━━━━━━━━━━━━━━━━━━ 423s 1s/step - accuracy: 0.1438 - loss: 4.1682 - val_accuracy: 0.1796 - val_loss: 4.0051 - learning_rate: 1.0000e-04
Epoch 3/50
389/389 ━━━━━━━━━━━━━━━━━━━━ 0s 971ms/step - accuracy: 0.1602 - loss: 4.0791
Epoch 3: val_loss improved from 4.00513 to 3.93235, saving model to best_dermnet_model.keras
389/389 ━━━━━━━━━━━━━━━━━━━━ 398s 1s/step - accuracy: 0.1574 - loss: 4.0767 - val_accuracy: 0.1934 - val_loss: 3.9324 - learning_rate: 1.0000e-04
Epoch 4/50
389/389 ━━━━━━

In [ ]:
# ---------------------------
# Evaluation
# ---------------------------
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

model.save("dermnet_skin_disease_model.keras")
# Later when running webcam
model = load_model("dermnet_skin_disease_model.keras")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data preparation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    "E:/VS/Project/DERMNET/train",    # <-- change path to your dermnet dataset
    target_size=(128, 128),
    color_mode="rgb",
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    "E:/VS/Project/DERMNET/train",
    target_size=(128, 128),
    color_mode="rgb",
    class_mode="categorical",
    subset="validation"
)

# ✅ Get label names directly from dataset
label_names = list(train_gen.class_indices.keys())
print("Detected classes:", label_names)

label_names = list(train_gen.class_indices.keys())



In [ ]:
disease_treatment = {
    "Acne and Rosacea Photos": "Use benzoyl peroxide, salicylic acid creams, or consult a dermatologist.",
    "Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions": "Seek immediate dermatologist consultation.",
    "Atopic Dermatitis Photos": "Apply moisturizers, hydrocortisone cream for flare-ups.",
    "Bullous Disease Photos": "Consult dermatologist, may require corticosteroid treatment.",
    "Cellulitis Impetigo and other Bacterial Infections": "Antibiotics may be required. Visit a doctor immediately.",
    "Eczema Photos": "Moisturize frequently, avoid harsh soaps, apply topical steroids if prescribed.",
    "Exanthems and Drug Eruptions": "Stop suspected drug, seek urgent medical care.",
    "Hair Loss Photos Alopecia and other Hair Diseases": "Minoxidil, hair serums, consult dermatologist for evaluation.",
    "Herpes HPV and other STDs Photos": "Antiviral treatment may be required. Seek medical advice.",
    "Light Diseases and Disorders of Pigmentation": "Use sunscreen, depigmenting creams, dermatology consultation.",
    "Lupus and other Connective Tissue diseases": "Needs rheumatologist/dermatologist care, avoid sunlight.",
    "Melanoma Skin Cancer Nevi and Moles": "Possible skin cancer. Seek urgent dermatologist consultation.",
    "Nail Fungus and other Nail Disease": "Antifungal creams or oral antifungals may be required.",
    "Poison Ivy Photos and other Contact Dermatitis": "Wash skin, apply calamine lotion, antihistamines.",
    "Psoriasis pictures Lichen Planus and related diseases": "Moisturizers, coal tar, corticosteroid creams.",
    "Scabies Lyme Disease and other Infestations and Bites": "Apply permethrin cream, wash clothing/bedding.",
    "Seborrheic Keratoses and other Benign Tumors": "Usually harmless, consult dermatologist if changing.",
    "Systemic Disease": "Skin symptoms linked to internal disease. Seek full medical evaluation.",
    "Tinea Ringworm Candidiasis and other Fungal Infections": "Antifungal creams or oral antifungal medications.",
    "Urticaria Hives": "Take antihistamines, avoid allergens/triggers.",
    "Vascular Tumors": "Consult dermatologist, treatment depends on severity.",
    "Vasculitis Photos": "Requires urgent medical attention, possible immunosuppressive therapy.",
    "Warts Molluscum and other Viral Infections": "Salicylic acid, cryotherapy, consult dermatologist."
}
for cls in label_names:
    if cls not in disease_treatment:
        disease_treatment[cls] = "Consult a dermatologist for treatment advice."


In [ ]:
# ---------------------------
# Performance Visualization
# ---------------------------
# Accuracy/Loss curves
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.legend()
plt.title("Accuracy over Epochs")

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend()
plt.title("Loss over Epochs")
plt.show()

In [ ]:
# Confusion Matrix
y_pred = np.argmax(model.predict(test_images), axis=-1)
y_true = np.argmax(test_labels, axis=-1)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=label_names, yticklabels=label_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
# Classification Report
print(classification_report(y_true, y_pred, target_names=label_names))

In [ ]:
# Per-class Accuracy
class_acc = np.diag(cm) / np.sum(cm, axis=1)
plt.figure(figsize=(12, 6))
plt.bar(label_names, class_acc, color="teal")
plt.xticks(rotation=90)
plt.title("Accuracy per Class")
plt.show()

In [ ]:
import cv2
import numpy as np
import pyttsx3
import time
import webbrowser

# Initialize TTS
engine = pyttsx3.init()

def predict_disease(frame, threshold=0.3):
    img = cv2.resize(frame, (128, 128))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)
    confidence = np.max(prediction)
    pred_class = np.argmax(prediction, axis=-1)[0]
    pred_label = label_names[pred_class]

    if confidence >= threshold:
        return pred_label, confidence
    return None, confidence

def speak_suggestion(disease):
    if disease in disease_treatment:
        suggestion = disease_treatment[disease]
    else:
        suggestion = "No treatment suggestion available."
    engine.say(f"The detected condition is {disease}. Suggested treatment: {suggestion}")
    engine.runAndWait()
    return suggestion

disease_treatment = {
    "Acne and Rosacea Photos": {
        "advice": "Use benzoyl peroxide or salicylic acid creams.",
        "link": "https://www.1mg.com/search/all?name=benzoyl%20peroxide%20cream"
    },
    "Scabies Lyme Disease and other Infestations and Bites": {
        "advice": "Apply permethrin cream, wash clothing/bedding.",
        "link": "https://www.1mg.com/search/all?name=permethrin%20cream"
    },
    "Psoriasis pictures Lichen Planus and related diseases": {
        "advice": "Coal tar shampoo or corticosteroid creams.",
        "link": "https://www.1mg.com/search/all?name=coal%20tar%20cream"
    },
    "Melanoma Skin Cancer Nevi and Moles": {
        "advice": "Possible skin cancer. Seek urgent dermatologist consultation.",
        "link": "https://www.google.com/search?q=melanoma+dermatologist"
    },
}

# ---------------------------
# Webcam Loop
# ---------------------------
cap = cv2.VideoCapture(0)
last_prediction = None
last_pred_time = time.time()
last_speak_time = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    display_frame = cv2.resize(frame, (640, 480))
    pred_label, confidence = predict_disease(display_frame)

    if pred_label:
        label_text = f"{pred_label} ({confidence*100:.1f}%)"
        cv2.putText(display_frame, label_text, (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if pred_label in disease_treatment:
            cv2.putText(display_frame, disease_treatment[pred_label],
                        (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (255, 255, 255), 2)

        # Speak suggestion every 5 sec
        if time.time() - last_speak_time > 5:
            speak_suggestion(pred_label)
            last_speak_time = time.time()

        # Auto open Google after 5 sec of same prediction
        if pred_label == last_prediction and time.time() - last_pred_time > 5:
            query = f"{pred_label} treatment"
            webbrowser.open(f"https://www.google.com/search?q={query}")
            last_pred_time = time.time()

        last_prediction = pred_label
    else:
        cv2.putText(display_frame, "No confident prediction",
                    (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Skin Disease Detection", display_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
